# Generic Template for a Basic Encrypted Order Book (via Sorted Arrays)

An *encrypted order book* is an [order book](https://en.wikipedia.org/wiki/Order_book) in which orders are submitted in encrypted form and remain encrypted until they are (partially or fully) revealed as part of a transaction. This document presents a generic template for an encrypted order book that relies on maintaining sorted arrays of orders submitted by clients. An encrypted order book implementation can be built based on this template by providing implementations of the base functions (for example, using a secure multi-party computation or homomorphic encryption scheme) that can operate on encrypted data.

## Introduction

The implementation template presented in this document relies on data structures and algorithms that are chosen based on their compatibility with secure computation schemes and protocols such as secure multi-party computation (MPC). When these data structures and algorithms are implemented using such schemes or protocols, the encrypted order book possesses certain privacy features. However, these features also introduce some costs and limitations.

**Implementation Overview.** The implementation presented in this document is described briefly below. This description focuses on the functional (rather than cryptographic) parts of the implementation.

1. The *operator* of the encrypted order book initializes and maintains two sorted arrays of a fixed size $k$ of orders: one for asks and one for bids. The initialized arrays are filled with *null* orders (*i.e.*, placeholder orders that do not match any other order).

2. When an individual *client* submits an *order* (a proposed *transaction* specifying a price and quantity):

    a. the order is added as the first entry in *both* arrays (overwriting the entries in those positions);

    b. both arrays are sorted by swapping contiguous pairs of entries (from the bottom to the top) such that null orders are always demoted and orders with the highest (for bids) or lowest (for asks) price are promoted;

    c. the top array entries (one from each array) are compared to determine if a *transaction* can be executed; and

    d. the top array entries are updated and processing resumes at step (b) above (if a transaction was executed) or no further processing occurs.

4. Once exactly $k$ orders have been submitted, the order book does not accept any further orders.

**Privacy Features.** If individual orders are encrypted and the functions in the implementation that process orders are implemented using a compatible secure computation scheme, the encrypted order book ensures that (over the course of the order book's operation) the operator learns *only* (1) the number of orders submitted by each client and (2) the executed transactions (including their quantity and price). Notably, the operators *does not* learn any information about an order (*i.e.*, whether it is a bid or an ask, its price, or its quantity) if it does not contribute to a transaction, and also has no way to detect whether a transaction partially or fully fulfilled the orders on either side.

**Limitations and Costs.** The most significant limitation of an implementation is that the number of orders $k$ that can be submitted must be fixed in advance and cannot be increased. The most significant computation costs (which may also include communication costs in the case of MPC protocols) are:

1. $O(k)$ comparison and conditional operations must be performed after each submission of an order, and

2. $O(k)$ *additional* comparison and conditional operations must be performed for *every* transaction that is detected and executed as a consequence of an order submission.

## Data Structures and Functions

The data structures and functions below are reference implementations that represent and operate on data in the clear. In order to implement an encrypted order book, only the [core functions](#core-functions-on-orders) must be implemented in such a way that they can be applied to encrypted inputs. The [order book data structure and methods](#order-book-implementation) are themselves cryptography-agnostic (albeit peculiar in their implementation). They can inherit all of their cryptographic features from the core functions (if the core functions operate on encrypted values).

### Transaction and Order Data Structures<a id='data-structures'></a>

A *transaction* consists of an integer price and an integer quantity. An *order* is a proposed transaction submitted by an individual client. An order may be an *ask* or a *bid* order, and this is encoded in its ``category`` attribute (where ``-1`` indicates it is an ask order and ``1`` indicates it is a bid order). The client identity and the commodity/security associated with an order or transaction is assumed to be tracked in some manner that falls outside the scope of the implementation template presented in this article.

In [1]:
from __future__ import annotations

class transaction:
    """
    Data structure for individual transactions (consisting of an integer
    price and an integer quantity).
    """
    def __init__(self, price: int, quantity: int):
        self.price: int = price
        self.quantity: int = quantity

    def __repr__(self: order) -> str:
        """
        Return a human-readable and ``eval``-friendly representation.
        """
        return 'transaction' + str((self.price, self.quantity))

class order:
    """
    Data structure for an order (*i.e.*, a proposed transaction).
    """
    def __init__(self, price: int, quantity: int, category: int):
        self.price: int = price
        self.quantity: int = quantity
        self.category: int = category # ``-1`` for asks, ``1`` for bids.
    
    def __repr__(self: order) -> str:
        """
        Return a human-readable and ``eval``-friendly representation.
        """
        return 'order' + str((self.price, self.quantity, self.category))

### Primitive Operations for Simple Types<a id='primitive-operations-for-simple-types'></a>

To implement an encrypted order book, every [core function](#core-functions-on-orders) must be implemented using a secure computation scheme. The base functions can be implemented either (1) via composition of primitive operations that can be applied to encrypted integer and boolean values or (2) as high-level, black-box functions (*e.g.*, custom-built MPC protocols) that be applied to encrypted orders. To accommodate scenario (1), this section provides a comprehensive collection of primitive operations (in the form of reference implementations that operate on plaintext integer and boolean values). For completeness, these reference implementations are also used within the reference implementations of the [core functions](#core-functions-on-orders) (both directly and via [primitive operations for non-simple types](#primitive-operations-for-non-simple-types)).

In [2]:
def or_(a: bool, b: bool) -> bool:
    """
    Returns the disjunction of the two boolean arguments.
    """
    return a or b

def if_then_else(c: bool, n: int, m: int) -> int:
    """
    Ternary operator that returns the second argument if the first is true,
    and otherwise returns the third argument.
    """
    return n if c else m

def equal_to_zero(n: int) -> bool:
    """
    Returns a boolean that represents whether the argument is equal to zero.
    """
    return n == 0

def less_than(n: int, m: int) -> bool:
    """
    Returns a boolean value that represents whether the first argument is
    strictly less than the second argument. 
    """
    return n < m

def plus(n: int, m: int) -> int:
    """
    Return the integer result of adding the second argument
    from the first argument.
    """
    return n + m

def minus(n: int, m: int) -> int:
    """
    Return the integer result of subtracting the second argument
    from the first argument.
    """
    return n - m

The two primitive operations below can be defined in terms of the operations above (although this is not required if more specialized alternative implementations are available and preferred).

In [3]:
def minimum(n: int, m: int) -> int:
    """
    Return the smaller of the two arguments.
    """
    return if_then_else(less_than(n, m), n, m)

def maximum(n: int, m: int) -> int:
    """
    Return the larger of the two arguments.
    """
    return if_then_else(less_than(n, m), m, n)

The primitive operations below are used in this implementation template to signify when a value must be decrypted.

In [4]:
from typing import Union

def decrypt_bool(b: bool) -> bool:
    """
    Decrypt the encrypted boolean value.
    """
    return b

def decrypt_int(n: int) -> int:
    """
    Decrypt the encrypted integer.
    """
    return n

### Primitive Operations for Non-Simple Types<a id='primitive-operations-for-non-simple-types'></a>

It is natural to define variants of [primitive operations defined on simple types](#primitive-operations-for-simple-types) for more complex types (such as tuples) and/or to user-defined types (such as [transactions and orders](#data-structures)). The primitive operations below are defined for instances of the [order data structure](#data-structures) and pairs thereof (using only [primitive operations defined on simple types](#primitive-operations-for-simple-types)).

In [5]:
def if_then_else_order(c: bool, s: order, t: order) -> order:
    """
    Ternary operator that returns the second argument if the first is true,
    and otherwise returns the third argument.
    """
    price = if_then_else(c, s.price, t.price)
    quantity = if_then_else(c, s.quantity, t.quantity)
    category = if_then_else(c, s.category, t.category)
    return order(price, quantity, category)

def minimum_order(s: order, t: order) -> order:
    """
    Return the order with the lower price.
    """
    condition = less_than(s.price, t.price)
    price = if_then_else(condition, s.price, t.price)
    quantity = if_then_else(condition, s.quantity, t.quantity)
    category = if_then_else(condition, s.category, t.category)
    return order(price, quantity, category)

def maximum_order(s: order, t: order) -> order:
    """
    Return the order with the higher price.
    """
    condition = less_than(s.price, t.price)
    price = if_then_else(condition, t.price, s.price)
    quantity = if_then_else(condition, t.quantity, s.quantity)
    category = if_then_else(condition, t.category, s.category)
    return order(price, quantity, category)

def if_then_else_pair(
        c: bool,
        p: tuple[order, order],
        q: tuple[order, order]
    ) -> tuple[order, order]:
    """
    Ternary operator that returns the second argument if the first is true,
    and otherwise returns the third argument.
    """
    return (
        if_then_else(c, p[0], q[0]),
        if_then_else(c, p[1], q[1])
    )

If a language and compiler offer a more rich collection of features, polymorphic primitive operations corresponding to those above (that can be applied to tuples or user-defined record types) might be made available. Below is an example of a polymorphic ternary operator that can be used with any two iterable objects.

In [6]:
from typing import Iterable

def if_then_else_iterable(c: bool, xs: Iterable, ys: Iterable) -> Iterable:
    """
    Ternary operator that returns the second argument if the first is true,
    and otherwise returns the third argument.
    """
    return (if_then_else(c, x, y) for (x, y) in zip(xs, ys))

### Core Functions on Transactions<a id='core-functions-on-transactions'></a>

To implement an encrypted order book, every function in this section must be implemented using a secure computation scheme (such that the function can be applied to pairs of encrypted transactions). This can either be accomplished by (1) leveraging primitive operations on [simple](#primitive-operations-for-simple-types) and [non-simple](#primitive-operations-for-non-simple-types) types that can be applied to encrypted values or (2) implementing the functions directly as black boxes using some other approach.

Recall that an *order* is a proposed transaction submitted by a client. Orders are sorted according to price, except that orders that have a ``quantity`` and/or ``category`` of ``0`` always appear earlier (regardless of what their price attribute may be). The latter requirement ensures that placeholder orders, fully executed orders, and orders that belong to the wrong category (for that specific array) can never lead to a transaction.

In order to implement an encrypted order book, it must be possible to evaluate the ``ascending`` and ``descending`` functions presented below on pairs of encrypted orders. These functions perform a conditional swap of an input pair of orders; they are used to maintain [encrypted sorted arrays](#sorted-array-data-structure) of orders.

In [7]:
from typing import Callable

def descending(pair: tuple[order, order]) -> tuple[order, order]:
    """
    Return a tuple in which the two orders in the supplied tuple are arranged
    in descending order by price, except that any order with a ``quantity`` of
    ``0`` or a category that is not ``0`` appears earlier. Effectively, the
    below algorithm is implemented using primitive operations:

        (s, t) = pair
        if s.quantity == 0 or s.category == 0:
            return (s, t)
        if t.quantity == 0 or t.category == 0:
            return (t, s)
        return (maximum_order(s, t), minimum_order(s, t))
    """
    (s, t) = pair
    return if_then_else_pair(
        or_(equal_to_zero(t.quantity), equal_to_zero(t.category)),
        (t, s),
        if_then_else_pair(
            or_(equal_to_zero(s.quantity), equal_to_zero(s.category)),
            (s, t),
            (maximum_order(s, t), minimum_order(s, t))
        )
    )

def ascending(pair: tuple[order, order]) -> tuple[order, order]:
    """
    Return a tuple in which the two orders in the supplied tuple are arranged
    in ascending order by price, except that any order with a quantity of zero
    appears earlier.
    """
    (s, t) = pair
    return if_then_else_pair(
        or_(equal_to_zero(t.quantity), equal_to_zero(t.category)),
        (t, s),
        if_then_else_pair(
            or_(equal_to_zero(s.quantity), equal_to_zero(s.category)),
            (s, t),
            (minimum_order(s, t), maximum_order(s, t))
        )
    )

In order to implement an encrypted order book, it must be possible to evaluate the ``match`` and ``update`` functions defined below on pairs of encrypted orders. These are used by the [order book implementation](#order-book-implementation) to determine whether two orders can lead to an executed transaction, and to update the top entries in the order book's two sorted arrays in a way that reflects the effects of that executed transaction.

In [8]:
from typing import Optional

def match(ask_bid: tuple[order, order]) -> Optional[transaction]:
    """
    Return a transaction that can be executed (if that is possible) or ``None``
    (if that is not possible). Note that a transaction does not necessarily
    exhaust both orders.
    """
    (ask, bid) = ask_bid

    # If a transaction is possible, the price and quantity of that
    # transaction would be defined as below. Note that the below code
    # must be executed every time within a secure computation scheme
    # or protocol (as the value of ``no_match`` depends on it).
    # Such a flexibility may or may not be availabe (depending on the
    # specific use case on the security/privacy features that must be
    # supported).
    price = ask.price
    quantity = minimum(ask.quantity, bid.quantity)

    # Compute an encrypted boolean value that indicates whether a
    # transaction is possible.
    no_match = or_(
        less_than(bid.price, ask.price),
        or_(
            equal_to_zero(quantity),
            or_(
              equal_to_zero(ask.category),
              equal_to_zero(bid.category)
            )
        )
    )

    # The value of ``no_match`` is decrypted. If it is ``False``,
    # then the ``price`` and ``quantity`` values are decrypted.
    # The below statement may require specialized language or secure
    # computation framework features that allow programmers to
    # conditionally decrypt.
    return (
        None
        if decrypt_bool(no_match) else # This is an unconditional decryption.
        transaction(
            decrypt_int(price), # This is a conditional decryption.
            decrypt_int(quantity) # This is a conditional decryption.
        )
    )

def update(ask_bid: tuple[order, order]) -> tuple[order, order]:
    """
    Return a pair that either (1) is identical to the input pair because
    no transaction is possible or (2) is updated to reflect that the
    transaction that is possible has been executed.

    Note that because the ``match`` method is the only way that the
    order book can observe anything about the array data, this method
    does not need to perform additional checks. If a transaction is not
    possible because the ask or bid (or both) have a ``quantity`` that is
    ``0`` or a ``category`` that is not ``0``, the updates performed in
    this function can never affect a decrypted output.
    """
    (ask, bid) = ask_bid
    quantity = minimum(ask.quantity, bid.quantity)
    return if_then_else_pair(
        less_than(bid.price, ask.price),
        ask_bid, # No change.
        ( # Return orders that reflect an executed transaction.
            order(ask.price, minus(ask.quantity, quantity), ask.category),
            order(bid.price, minus(bid.quantity, quantity), bid.category)
        )
    )

### Sorted Array Data Structure<a id="sorted-array-data-structure"></a>

A sorted array maintains orders in either an ascending or a descending arrangement according to a specific conditional swap function (that operates on one pair of orders at a time).

In [9]:
from typing import Callable

class array(list):
    """
    Arrays of orders (maintained in ascending or descending order).
    """
    def arrange(
            self: array,
            conditional_swap: Callable[[tuple[order, order]], tuple[order, order]]
        ):
        """
        Sort this instance such that every pair is ordered according to the
        supplied conditional swap function.
        """
        for i in range(len(self) - 1):
            pair = (self[i], self[i + 1])
            (x, y) = conditional_swap(pair)
            self[i] = x
            self[i + 1] = y

    def inject(
            self: array,
            order: transaction,
            conditional_swap: Callable[[tuple[order, order]], tuple[order, order]]
        ):
        """
        Add a new entry and sort the array.
        """
        self[0] = order
        self.arrange(conditional_swap)

The example below tests the ``array`` data structure. Note that orders with a ``quantity`` of ``0`` always appear earlier in an ``array`` instance (while all other entries are sorted according to the supplied conditional swap function).

In [10]:
from random import randint, seed

oo: float = float('inf') # To fill array with initial null ask orders.
seed(-123)
length = 6

orders = [order(randint(4, 9), randint(0, 1), 0) for _ in range(length)]
asks = array([order(oo, 0, 1) for _ in range(length)])
for o in orders:
    asks.inject(o, descending)

orders = [order(randint(1, 6), randint(0, 1), 0) for _ in range(length)]
bids = array([order(0, 0, 1) for _ in range(length)])
for o in orders:
    bids.inject(o, ascending)

print(asks)
print(bids)

[order(4, 1, 0), order(4, 1, 0), order(6, 0, 0), order(4, 1, 0), order(8, 1, 0), order(6, 0, 0)]
[order(2, 0, 0), order(3, 1, 0), order(6, 0, 0), order(2, 0, 0), order(4, 0, 0), order(5, 1, 0)]


## Order Book Implementation<a id='order-book-implementation'></a>

The order book implementation maintains two sorted arrays (one for asks and one for bids). When a new order is added, the below steps are performed.

1. The order is injected into both arrays (adding ``1`` to the ``category`` attribute before injecting into the array of asks, and adding ``-1`` to the ``category`` attribute before injecting into the array of bids).

2. The top entries (*i.e.*, last entries at index ``-1``) in the two arrays are processed using the ``match`` and ``update`` functions.

3. If a transaction is possible, the arrays are arranged again and control is returned to step (2) above (in case there are new transactions possible given the rearranged arrays).

The ``clear`` method implements the above steps and returns a list of *transactions*. A *transaction* consists of an execution price (*i.e.*, the asking price in a matching bid-ask pair of orders) and the quantity involved (*i.e.*, the minimum of the ask order quantity and the bid order quantity). 

In [11]:
from typing import Optional

class book:
    """
    An order book that allows the submission of bid and ask orders,
    as well as the matching of orders in order to identify transactions.
    """
    def __init__(self: book, size: int):
        """
        Initialize an order book of the specified size. No more
        bid orders can be submitted after ``size`` bid orders have
        been submitted (and likewise for ask orders).
        """
        self.asks = array([order(oo, 0, 0) for _ in range(size)])
        self.bids = array([order(0, 0, 0) for _ in range(size)])

    def order(self: book, order: order):
        """
        Accept and store a submitted order (and sort the arrays).
        """
        # If a bid order is submitted, then adding ``-1`` to its ``category``
        # attribute ensures that its ``category`` attribute is ``0`` (and,
        # thus, that it will be sorted below other orders).
        ask = type(order)(order.price, order.quantity, plus(order.category, -1))
        self.asks.inject(ask, descending)

        # If an ask order is submitted, then adding ``1`` to its ``category``
        # attribute ensures that its ``category`` attribute is ``0`` (and,
        # thus, that it will be sorted below other orders).
        bid = type(order)(order.price, order.quantity, plus(order.category, 1))
        self.bids.inject(bid, ascending)

    def match(self: book) -> Optional[order]:
        """
        Update the last entries of the ask and bid arrays using
        the matching function. Then, sort both arrays to move any
        fully exhausted orders below non-exhausted orders.
        """
        ask = self.asks[-1]
        bid = self.bids[-1]
        transaction = match((ask, bid))

        if transaction is not None:
            (ask, bid) = update((ask, bid))
            self.asks[-1] = ask
            self.bids[-1] = bid
            self.asks.arrange(descending)
            self.bids.arrange(ascending)

        return transaction

    def clear(self: book) -> list[order]:
        """
        Obtain transactions until no more are possible, and return a
        list containing all the transactions (in the order they were
        obtained).
        """
        transactions = []
        while True:
            transaction = self.match()
            if transaction is None:
                break
            transactions.append(transaction)

        return transactions

## Logs and Simulations

In this section, a *log* is defined to be an ordered sequence of orders and transactions. An *order log* is a log that contains only ask and bid orders (*e.g.*, that might be submitted over time by participants in an exchange). The function below generates a random order log of a specified length.

In [12]:
from random import randint, choice, seed

def random_order_log(seed_value: int, length: int) -> list[order]:
    """
    Generate a random order log of the specified length using the
    supplied seed value.
    """
    seed(seed_value)
    return [
        order(randint(1, 9) * 100, randint(1, 9), choice([-1, 1]))
        for _ in range(length)
    ]

An *simulated log* is a log that contains ask and bid orders interspersed with transactions (where each transaction appears as early as possible and has the maximum quantity possible). A *simulation* is a function from an order log to a simulated log.

In [13]:
from random import randint, choice, seed

def simulation(order_log: list[order]) -> list[Union[order, transaction]]:
    """
    Return the simulated log corresponding to the supplied order log.
    """
    b = book(len(order_log)) # Every order is added to both arrays.

    simulated_log = []
    for o in order_log:
        simulated_log.append(o)
        b.order(o)
        for transaction in b.clear():
            simulated_log.append(transaction)

    return simulated_log

The example below tests the ``book`` data structure and its methods by performing a simulation for a random order log.

In [14]:
order_log = random_order_log(1, 7)
for entry in simulation(order_log):
    print(entry)

order(300, 2, 1)
order(200, 8, 1)
order(800, 7, -1)
order(200, 8, -1)
transaction(200, 2)
transaction(200, 6)
order(700, 7, -1)
order(800, 5, -1)
order(200, 6, -1)
transaction(200, 2)


In [15]:
# End of file.